<a href="https://colab.research.google.com/github/davidsantos5092/Aplicada/blob/main/Taller_RBPP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Practica 1: Control de LED con Botón**

 **Juan David Santos Poblador--Cod: 20222005092**

## **Introducción**


En esta práctica, aprenderemos a controlar el parpadeo de un LED utilizando un botón. Exploraremos 3 formas distintas de programación en la Raspberry Pi Pico (W) usando Micropython.

### **Materiales**








*   Raspberry Pi Pico (W)
*   LED


*   Botón

*   Resistencias (según necesidad)
*   Cables de conexión




  **Programación**



A continuación, presentamos tres enfoques distintos para hacer parpadear el LED mientras se presiona el botón.


**Código con while**





In [ ]:
#Descargamos las bibliotecas necesesarias para la Raspberry Pi Pico W
from machine import Pin #Con la libreria Machine se interactua con los pines

import time #Con la libreria time se buscan funciones relacionadas con tiempo

led_pin = Pin(0, Pin.OUT) #Se coloca el pin 0 como salida para el LED
button_pin = Pin(15, Pin.IN,Pin.PULL_UP)  #Se elige el pin 15 como pin de entrada para el botón con una resistencia de pull-up

while True: #inicio de un bucle infinito
    button_state = button_pin.value() #Hace una Lectura del estado del botón y lo guarda en la variable button_state
    if button_state == 0: #Verifica si el botón está presionado
        led_pin.toggle()  #Alterna el estado del LED, poniendolo en bajo o alto según el if anterior
        time.sleep(0.02) #el programa se para durante el período de tiempo antes de seguir con las indicaciones siguientes

**Código con IRQ**

In [ ]:
from machine import Pin,Timer #Importa clases tales comos pin y timer de la libreria machine, para controlar la salida de los pines y funciones de tiempo

pin_led = Pin(0, mode=Pin.OUT) #se pone el pin 0 como una salida para el LED
pin_button = Pin(15, mode=Pin.IN, pull=Pin.PULL_UP) #se pone el pin 15 como una entrada para el botón con resistencia y se crea un objeto pin_button para interactuar con el botón

def toggle_led(t:Timer): #definimos la función que se activará cuando se conecte el timer (temporizador)
    pin_led(not pin_led() and pin_button()) #Se verifica si el botón está abajo y se invierte el estado actual del LED

Timer().init(mode=Timer.PERIODIC, period= 500, callback=toggle_led) # Timer().init(), está puesto un temporizador para que pueda  generar instrasmicencia cada 500 ms. Cada vez que ocurra una intrasmitencia, se relacionara a la función toggle_led, lo que provocara un cambio en las caracteristicas del LED.

**Código con PIO**

In [ ]:
import time #Con time se llaman las funciones relacionadas al tiempo
import rp2 #Se importa un módulo que da acceso al procesador de entrada y salida  de la Raspberry
from machine import Pin ##Con la clase Pin de la libreria Machine se manipula con los pines


@rp2.asm_pio(set_init=rp2.PIO.OUT_LOW) #está esta función antes de la definición de la función blink(), está configurando el estado inicial  de salida controlados por el PIO. Nos aseguramos de analizar los pines de salida para que se encuentren  bajo al inicio del programa

def blink(): #Instrucciones que controlan el titileo del LED
    wrap_target() #Etiqueta especial que señala el inicio del bucle principal del programa. Cuando se llegamos al final del bucle , el programa hace un intervalo de salto de nuevo a esta etiqueta para continuar el bucle
    jmp(pin, "one") #Aqui verifican  el estado del pin si es bajo (0). Si es así, tiene un intervalo de salto a la etiqueta "one". Si no, sigue con la siguiente orden
    set(pins, 1)   [31] #Esta orden da el estado de los pines de salida controlados por la PIO en 1 (alto). Los pines de salida se establecerán en alto durante un ciclo de reloj
    label("one") #Esta orden concreta una etiqueta llamada "one". La instrucción jmp  salta a esta etiqueta si el pin esta bajo
    nop()          [31] #Se utilizan múltiples instrucciones "nop" para introducir retrasos y controlar la velocidad del titileo del LED. [31] indica que se repitiran una cantidad de 31 veces
    nop()          [31]
    nop()          [31]
    nop()          [31]
    set(pins, 0)   [31] #Establece el estado de los pines del PIO en 0 (bajo)
    nop()          [31]
    nop()          [31]
    nop()          [31]
    nop()          [31]
    wrap() #Esta Orden marca el fin del bucle del programa y provoca que el programa vuelva al inicio del bucle

sm = rp2.StateMachine(0, blink, freq=2000, set_base=Pin(0),jmp_pin=Pin(15)) #ordena una máquina de estado (state machine) utilizando el PIO. Se utiliza la función blink() definida previamente. Se establece una frecuencia de 2000 Hz y se configuran los pines base y de salto utilizando los pines 0 y 15 respectivamente
sm.active(1) #Activa la state machine

## **Ejercicio de documentación**


1.   Comente los codigos.






2.   **¿Puede predecir si el LED parpadea al oprimir el botón o al soltar el botón?**

**Con While:** Si se percive el LED encenderse cuando se presiona el botón.

**En el caso de IRQ** presenta saltos por el temporizador. asi cuando  el temporizador se cierra cada 500 ms, se trae la función toggle_led(), que cambia el estado del LED. asi que cuando el temporizador se configura de manera periódica, la predicción es el LED cuando parpadeará en intervalos regulados y predichos.

**Con PIO** el LED titilara con una velocidad  alta. esta  sera constante y se podra predecir el titilar del LED ya que la velocidad de titileo se llama en las funciones establecidas en el código.








3. **Calcule el valor de la frecuencia en cada caso.**


En este código, el LED prende y apaga en un bucle while infinitamente cada vez que el botón es precionado. La cadencia en este caso dependerá de cuánto tiempo se mantiene presionado el botón, ya que el LED titilara mientras el botón esté presionado. No hay una cadencia constante predefinida en este caso, ya que está directamente relacionada con la acción del botón.

Código con IRQ:

En este código, el temporizador genera interrupciones con un período de 500 ms. Cada vez que ocurre una interrupción, se trae  la función toggle_led(), que cambia el estado del LED. La cadencia en este caso está determinada por el período del temporizador y se calcula como la inversa del período. En este caso, la cadencia sería:

**Cadencia = 1 / 0.5 = 2 Hz**









Esto significa que el LED titilara dos veces por segundo.

Código con PIO:

En este código, la función blink() tiene el control del titileo del LED utilizando el lenguaje de ensamblador del PIO. La cadencia se controla a través del tiempo que lleva a ejecutar las indicaciones dentro de la función blink(). Las instrucciones nop() introducen retrasos y controlan la velocidad del titileo. Dado que el lenguaje ensamblador del PIO es altamente eficaz y rápido, la cadencia se mantiene alta. La cadencia en este caso está relacionada con la velocidad de ejecución del ensamblador, y es posible que sea más alta que en los otros enfoques.







4.  **Determine los pines usados y realice el montaje.**
Para la práctica, se usaron los pines 0, 3 y 15. Tomando al pin 0 como la entrada del LED , el pin 3 se conecta a la tierra **GND** y el pin 15 se conecta a la entrada del interruptor.

5.  **Ejecute el código y mida la frecuencia y el error.**

Para el primer montaje de la practica, se uso una frecuencia  de 25 Hz, valor proveniente al conocer que la función sleep apaga el Led durante 0.02 seg lo que corresponde a la mitad del periodo total de la señal. Por ende al duplicar este valor y Sacar su inversa tendremos la frecuencia de 25 Hz. Contrastando esto con la frecuencia con el osciloscopio donde nos entrego el valor de 24.96 Hz. concluyendo un 0.16 de error.



6.   **Describa como puede modificar la frecuencia y cual sería la máxima en cada caso, debe consultar la el menor valor de period en Timer. El mayor vlor de freq en `StateMachine``es 1'250,000 Hz.**

Para ajustar la tasa de parpadeo, se utiliza la función time.sleep() con el objetivo de introducir un intervalo de inactividad entre los cambios de estado del LED.

Este intervalo de inactividad afecta directamente la tasa de parpadeo. Para modificar la tasa, lo más conveniente es cambiar el valor del intervalo de inactividad en la función time.sleep(). Este intervalo se expresa en segundos.

Al calcular la tasa máxima alcanzable, es esencial tener en cuenta tanto la velocidad de ejecución del código en el microcontrolador como el tiempo mínimo posible para el intervalo de inactividad. Si se desea que el LED parpadee a una tasa máxima teórica de 25 Hz, se puede calcular el intervalo de inactividad necesario de la siguiente manera:

Tasa = 25 Hz Intervalo = 1 / Tasa = 1 / 25 = 0.04 segundos (40 ms) Dado que time.sleep() toma argumentos en segundos, es necesario convertir el intervalo a segundos: T = 0.04 segundos.

No obstante, es fundamental tener en cuenta que este cálculo es teórico y depende tanto de la velocidad de ejecución del microcontrolador como de la eficacia del bucle while, por lo que en la práctica podría resultar complicado alcanzar una tasa tan alta debido a esas restricciones.

Es relevante destacar que, aunque la tasa máxima teórica puede ser alta, como 1,250,000 Hz, en la práctica, lograr esa tasa puede ser desafiante debido a limitaciones temporales y de procesamiento en el microcontrolador.

En resumen, se presume que este análisis es aplicable de manera general a todos los casos.


7.  **Modifique los programas para aumentar a la máxima frecuaencia y mida la frecuencia y el error.**


# **Concluciones**



1.  **Enumere las ventajas y deventajas de cada uno de los programas.**

El  programa **while** requiere de una configuración adicional, pero la cadencia de titileo depende del  tiempo que se mantenga presionado el botón.

  El  programa **IRQ**  tiene una interrupción para manipular el titileo, lo que permite una cadencia constante.

  El  programa **PIO** utiliza el procesador de entrada y salida programable , lo que tiene un control en el titileo.







2.   **Describa en que casos es conveniente cada uno de los programas.**


*  El  programa **while** se necesita cuando se quiere un titileo controlado  por el usuario.
*  El programa IRQ es útil cuando se quiere una cadencia constante de titileo y se quiere trabajar con interrupción.



3.  **¿Cual es la diferencia entre time y timer?**

**Time:**

*   El módulo time es una biblioteca estándar de Python que proporciona funciones relacionadas con el tiempo.
*  Se utiliza para medir el tiempo transcurrido, crear retrasos y manipular fechas y horas.

**Timer:**


*   "Timer" generalmente se refiere a un concepto más amplio que involucra el uso de la función time.time() u otras funciones de tiempo para medir el tiempo que lleva ejecutar una parte específica del código.
*   Los "timers" se utilizan para medir el rendimiento de una parte del código, identificar cuellos de botella y optimizar el rendimiento.





